In [1]:
%pylab inline
import xgboost as xgb
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
from sklearn.linear_model import LogisticRegression
import clustering as cl
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
df_actual = pd.read_csv("ms/auto_download/auto_download_files/germany-bundesliga-matches-2020-to-2021-stats.csv")

In [3]:
df_actual.index = df_actual.index + 1224

In [4]:
df_actual['goal_diff'] = df_actual['home_team_goal_count'] - df_actual['away_team_goal_count']

for index, row in df_actual[df_actual['status']=='complete'].iterrows():
    if df_actual['goal_diff'][index] > 0:
        df_actual.at[index,'result'] = 3
    elif df_actual['goal_diff'][index] == 0:
        df_actual.at[index,'result'] = 2
    else:
        df_actual.at[index,'result'] = 1

In [5]:
df_actual

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,...,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,goal_diff,result
1224,1600453800,Sep 18 2020 - 6:30pm,complete,NaN,Bayern München,Schalke 04,Felix Zwayer,0.0,0.0,3.0,...,24.50,1.04,1.20,1.53,2.20,1.95,1.77,Allianz Arena (München),8,3.0
1225,1600522200,Sep 19 2020 - 1:30pm,complete,NaN,Eintracht Frankfurt,Arminia Bielefeld,Benjamin Brand,0.0,0.0,1.0,...,4.60,1.18,1.59,2.50,4.40,1.57,2.20,Deutsche Bank Park (Frankfurt am Main),0,2.0
1226,1600522200,Sep 19 2020 - 1:30pm,complete,NaN,Union Berlin,Augsburg,Marco Fritz,0.0,0.0,0.0,...,3.30,1.37,2.15,3.95,7.75,1.83,1.83,Stadion An der Alten Försterei (Berlin),-2,1.0
1227,1600522200,Sep 19 2020 - 1:30pm,complete,NaN,Köln,Hoffenheim,Daniel Siebert,0.0,0.0,0.0,...,2.75,1.19,1.62,2.55,4.55,1.51,2.35,RheinEnergieStadion (Köln),-1,1.0
1228,1600522200,Sep 19 2020 - 1:30pm,complete,NaN,Werder Bremen,Hertha BSC,Sascha Stegemann,0.0,0.0,0.0,...,2.85,1.24,1.77,2.95,5.55,1.61,2.15,wohninvest WESERSTADION (Bremen),-3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,1621690200,May 22 2021 - 1:30pm,incomplete,NaN,Eintracht Frankfurt,Freiburg,NaN,1.0,3.0,1.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Deutsche Bank Park (Frankfurt am Main),0,NaN
1526,1621690200,May 22 2021 - 1:30pm,incomplete,NaN,Union Berlin,RB Leipzig,NaN,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Stadion An der Alten Försterei (Berlin),0,NaN
1527,1621690200,May 22 2021 - 1:30pm,incomplete,NaN,Köln,Schalke 04,NaN,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,RheinEnergieStadion (Köln),0,NaN
1528,1621690200,May 22 2021 - 1:30pm,incomplete,NaN,Werder Bremen,Borussia M'gladbach,NaN,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,wohninvest WESERSTADION (Bremen),0,NaN


In [6]:
from ms.database import MySQLDatabase

In [7]:
bot = MySQLDatabase()
df = bot.get('bookie')

Connecting to Database and geting results
    level_0  index       home_team_name       away_team_name  \
0      90.0   1224       Bayern München           Schalke 04   
1      91.0   1225  Eintracht Frankfurt    Arminia Bielefeld   
2      92.0   1226         Union Berlin             Augsburg   
3      93.0   1227                 Köln           Hoffenheim   
4      94.0   1228        Werder Bremen           Hertha BSC   
5      95.0   1229            Stuttgart             Freiburg   
6      96.0   1230    Borussia Dortmund  Borussia M'gladbach   
7      97.0   1231           RB Leipzig             Mainz 05   
8      98.0   1232            Wolfsburg     Bayer Leverkusen   
9       NaN   1233           Hertha BSC  Eintracht Frankfurt   
10      NaN   1234  Borussia M'gladbach         Union Berlin   
11      NaN   1235     Bayer Leverkusen           RB Leipzig   
12      NaN   1236             Mainz 05            Stuttgart   
13      NaN   1237             Augsburg    Borussia Dortmund  

In [8]:
df = df.drop_duplicates(subset=['index'],keep='last')

In [9]:
df_actual_rows = df_actual.loc[list(df['index'])]

In [10]:
l_result_last_game=list(df_actual_rows['result'])

In [11]:
df['real_result'] = l_result_last_game

In [12]:
df

,level_0,index,home_team_name,away_team_name,predicted_result,real_result,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,date_time
0,90.0,1224,Bayern München,Schalke 04,3.0,3.0,1.13,10.00,19.75,15:50:02 09/16/20 Europe
1,91.0,1225,Eintracht Frankfurt,Arminia Bielefeld,2.0,2.0,1.62,4.40,5.30,15:50:02 09/16/20 Europe
2,92.0,1226,Union Berlin,Augsburg,3.0,1.0,2.40,3.25,3.30,15:50:02 09/16/20 Europe
3,93.0,1227,Köln,Hoffenheim,1.0,1.0,2.65,3.70,2.65,15:50:02 09/16/20 Europe
4,94.0,1228,Werder Bremen,Hertha BSC,3.0,1.0,2.70,3.65,2.60,15:50:02 09/16/20 Europe
5,95.0,1229,Stuttgart,Freiburg,3.0,1.0,2.30,3.50,3.25,15:50:02 09/16/20 Europe
6,96.0,1230,Borussia Dortmund,Borussia M'gladbach,3.0,3.0,1.65,4.50,5.10,15:50:02 09/16/20 Europe
7,97.0,1231,RB Leipzig,Mainz 05,1.0,3.0,1.39,5.55,7.50,15:50:02 09/16/20 Europe
8,98.0,1232,Wolfsburg,Bayer Leverkusen,1.0,2.0,2.85,3.50,2.55,15:50:02 09/16/20 Europe
9,NaN,1233,Hertha BSC,Eintracht Frankfurt,1.0,NaN,2.30,3.60,3.10,14:06:21 09/24/20 CEST


In [13]:
# bot.write(df)

Connecting to Database
Successfully wrote to Database
